In [ ]:
/**************Clase pasada*********************/ 


proc import datafile='/folders/myfolders/RepasoI2/Clases y archivos/Clase 14/Lista_Alum.txt'
            out=mails_alu
            dbms=dlm
            replace;
       datarow=5;
     delimiter=';';
run;

%macro limpieza(datos);

Data &datos.;
set &datos.;
Rut2=SUBSTR(COMPRESS(Rut, ' .-'),1,8);
Dig_Rut=SUBSTR(COMPRESS(Rut, ' .-'),9,1);
Dig_Rut_ver=3*SUBSTR(COMPRESS(Rut, ' .-'),1,1)+2*SUBSTR(COMPRESS(Rut, ' .-'),2,1)+7*SUBSTR(COMPRESS(Rut, ' .-'),3,1)+
6*SUBSTR(COMPRESS(Rut, ' .-'),4,1)+5*SUBSTR(COMPRESS(Rut, ' .-'),5,1)+4*SUBSTR(COMPRESS(Rut, ' .-'),6,1)+
3*SUBSTR(COMPRESS(Rut, ' .-'),7,1)+2*SUBSTR(COMPRESS(Rut, ' .-'),8,1);
Dig_Rut_ver=mod(Dig_Rut_ver,11);
if Dig_Rut='K' then Dig_Rut=10;
CORR=1;
IF Dig_Rut^=Dig_Rut_ver then CORR=0;
Apellido_Materno=upcase(Apellido_Materno);
Apellido_Paterno=upcase(Apellido_Paterno);
Nombres=propcase(Nombres);
Correo=lowcase(Correo);
run;
data &datos.(keep=Apellido_Materno Apellido_Paterno Nombres Correo Rut2 Dig_Rut Corr);
set  &datos.;
run;

data Incorrectos;
set &datos.;
if Corr=1 then delete;
drop Corr;
run;
data Correctos;
set &datos.;
if Corr^=1 then delete;
drop Corr;
run;

ods pdf 
	style=minimal 
	file='/folders/myfolders/RepasoI2/Clases y archivos/Clase 14/Lista de Alumnos.pdf'; 
options nodate pageno=1 linesize=64 pagesize=60; 

title "Lista de Alumnos con rut Correcto"; 
proc print data=Correctos;run;
title "Lista de Alumnos con rut Incorrecto"; 
proc print data=Incorrectos;run;
title; 
ods pdf close;


%mend;


%limpieza(mails_alu);




/***********************************/  
/********   EJERCICIO 2   **********/
/***********************************/ 
%macro Curso(Reg=,Cur=,Nom=);
%Let Direccion=/folders/myfolders/RepasoI2/Clases y archivos/Clase 14/SIMCE/%upcase(&Cur.)/simce%lowcase(&Cur.)2013_region&Reg..xls;
PROC IMPORT DATAFILE="&Direccion."
	DBMS=XLS
	OUT=&Nom.;
	GETNAMES=YES;
RUN;
%mend;
%Curso(Reg=II,Cur=4B,Nom=Curso4B_II);



%macro Simce_Curso(Curso=);
%Curso(Reg=I,Cur=&Curso.,Nom=Curso&Curso._I);
%Curso(Reg=II,Cur=&Curso.,Nom=Curso&Curso._II);
%Curso(Reg=III,Cur=&Curso.,Nom=Curso&Curso._III);
%Curso(Reg=IV,Cur=&Curso.,Nom=Curso&Curso._IV);
%Curso(Reg=IX,Cur=&Curso.,Nom=Curso&Curso._IX);
%Curso(Reg=V,Cur=&Curso.,Nom=Curso&Curso._V);
%Curso(Reg=VI,Cur=&Curso.,Nom=Curso&Curso._VI);
%Curso(Reg=VII,Cur=&Curso.,Nom=Curso&Curso._VII);
%Curso(Reg=VIII,Cur=&Curso.,Nom=Curso&Curso._VIII);
%Curso(Reg=X,Cur=&Curso.,Nom=Curso&Curso._X);
%Curso(Reg=XI,Cur=&Curso.,Nom=Curso&Curso._XI);
%Curso(Reg=XII,Cur=&Curso.,Nom=Curso&Curso._XII);
%Curso(Reg=XIII,Cur=&Curso.,Nom=Curso&Curso._XIII);
%Curso(Reg=XIV,Cur=&Curso.,Nom=Curso&Curso._XIV);
%Curso(Reg=XV,Cur=&Curso.,Nom=Curso&Curso._XV);

Data Curso&Curso.;
SET Curso&Curso._I Curso&Curso._II Curso&Curso._III Curso&Curso._IV Curso&Curso._IX Curso&Curso._V
Curso&Curso._VI Curso&Curso._VII Curso&Curso._VIII Curso&Curso._X Curso&Curso._XI Curso&Curso._XII
Curso&Curso._XIII Curso&Curso._XIV Curso&Curso._XV;
KEEP IDREGION RBD ALUMNOS PROM_LEN;
RUN;
%mend;

%MACRO UNIR();
%Simce_Curso(Curso=2B);
%Simce_Curso(Curso=4B);
%Simce_Curso(Curso=6B);
%Simce_Curso(Curso=8B);
%Simce_Curso(Curso=2M);
PROC SORT DATA=CURSO2B;BY RBD;RUN;
PROC SORT DATA=CURSO4B;BY RBD;RUN;
PROC SORT DATA=CURSO6B;BY RBD;RUN;
PROC SORT DATA=CURSO8B;BY RBD;RUN;
PROC SORT DATA=CURSO2M;BY RBD;RUN;
Data SIMCE;
MERGE CURSO2B CURSO4B CURSO6B CURSO8B CURSO2M;
BY RBD;
%mend;

%UNIR();


%MACRO TABLA(Datos);
proc sort data=&Datos.; by RBD;run;
ods pdf 
	style=minimal 
	file='/folders/myfolders/RepasoI2/Clases y archivos/Clase 14/tablas_SIMCE.pdf'; 
options nodate pageno=1 linesize=64 pagesize=60; 
title "Cargos para clientes con 2 o menos productos"; 
proc means data=&Datos.;
var PROM.LECT;
by IDREGION;
output out=TABLA_SIMCE mean=Prom N=Freq std=Varianza;
run;
title; 
ods pdf close;
%MEND;

%TABLA(SIMCE);